<a href="https://colab.research.google.com/github/pabloperfer/CUDAexercises/blob/main/MatrixTranspositionwithSharedMemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numba.cuda.stubs import syncthreads
import numpy as np
from numba import cuda
import numba.types

TILE_DIM=32

@cuda.jit
def transpose(a_in, a_out):
  #Explicitly calculate indices rather than using cuda.grid(2)
  tile = cuda.shared.array((TILE_DIM, TILE_DIM), numba.types.int32)
  row = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
  col = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y
  
  
  for z in range (0,TILE_DIM,TILE_DIM):
    tile[cuda.threadIdx.y + z,cuda.threadIdx.x] = a_in[col + z, row ]
                                         
  cuda.syncthreads()
 
  for i in range (0,TILE_DIM,TILE_DIM):
    a_out[col + i , row] = tile[cuda.threadIdx.x][cuda.threadIdx.y + i]
    
  


In [2]:
size = 16384
a_in = cuda.to_device(np.arange(size*size,dtype=np.int32).reshape((size,size)))
a_out = cuda.device_array_like(a_in)

print(a_in.copy_to_host())
print(a_out.copy_to_host())

[[        0         1         2 ...     16381     16382     16383]
 [    16384     16385     16386 ...     32765     32766     32767]
 [    32768     32769     32770 ...     49149     49150     49151]
 ...
 [268386304 268386305 268386306 ... 268402685 268402686 268402687]
 [268402688 268402689 268402690 ... 268419069 268419070 268419071]
 [268419072 268419073 268419074 ... 268435453 268435454 268435455]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [3]:
threads_per_block=(32,32)
blocks_per_grid=(int(size/threads_per_block[0]),int(size/threads_per_block[1]))

In [4]:
timeit transpose[blocks_per_grid, threads_per_block](a_in,a_out); cuda.synchronize();print(a_out.copy_to_host())

[[        0     16384     32768 ...    491488    507872    524256]
 [        1     16385     32769 ...    491489    507873    524257]
 [        2     16386     32770 ...    491490    507874    524258]
 ...
 [267911197 267927581 267943965 ... 268402685 268419069 268435453]
 [267911198 267927582 267943966 ... 268402686 268419070 268435454]
 [267911199 267927583 267943967 ... 268402687 268419071 268435455]]
[[        0     16384     32768 ...    491488    507872    524256]
 [        1     16385     32769 ...    491489    507873    524257]
 [        2     16386     32770 ...    491490    507874    524258]
 ...
 [267911197 267927581 267943965 ... 268402685 268419069 268435453]
 [267911198 267927582 267943966 ... 268402686 268419070 268435454]
 [267911199 267927583 267943967 ... 268402687 268419071 268435455]]
[[        0     16384     32768 ...    491488    507872    524256]
 [        1     16385     32769 ...    491489    507873    524257]
 [        2     16386     32770 ...    491490    5